In [11]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import time

frames_folder = 'images/train'
empty_parking_folder = 'final_base'
output = 'output'

In [4]:
def perspective_transform(image, src_points, dst_points):
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    warped = cv2.warpPerspective(image, matrix, (image.shape[1], image.shape[0]))
    return warped

In [5]:
def apply_best_blur(image, method="gaussian", kernel_size=3, sigma=0, diameter=9, sigma_color=200, sigma_space=200):
    if method == "gaussian":
        blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)
    elif method == "median":
        blurred_image = cv2.medianBlur(image, kernel_size)
    elif method == "bilateral":
        blurred_image = cv2.bilateralFilter(image, diameter, sigma_color, sigma_space)
    else:
        raise ValueError("incorrect method")
    
    return blurred_image

In [6]:
def show_in_row(list_of_images, titles = None, disable_ticks = False):
    count = len(list_of_images)
    for idx in range(count):
        subplot = plt.subplot(1, count, idx+1)
        if titles is not None:
            subplot.set_title(titles[idx])

        img = list_of_images[idx]
        cmap = 'gray' if (len(img.shape) == 2 or img.shape[2] == 1) else None
        subplot.imshow(img, cmap=cmap)
        if disable_ticks:
            plt.xticks([]), plt.yticks([])
    plt.show()

In [22]:
def mse_match(imageA, imageB):
    err = np.mean((imageA.astype("float") - imageB.astype("float")) ** 2)
    
    return err <= 1200.0

def images_match(img1, img2, method=cv2.TM_CCOEFF_NORMED, threshold=0.8):
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

    res = cv2.matchTemplate(img2_gray, img1_gray, method)
    min_val, max_val, _, _ = cv2.minMaxLoc(res)

    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        match_value = min_val
        # For these methods, lower values indicate a better match
        match = match_value <= (1 - threshold)
    else:
        match_value = max_val
        # For these methods, higher values indicate a better match
        match = match_value >= threshold

    return match

def images_match_ncc(img1, img2, threshold=0.2):
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

    ncc = np.corrcoef(img2_gray, img1_gray)

    return ncc[0, -1] >= threshold

In [ ]:
parking_spaces = np.float32([
    [[121, 74], [130, 73], [116, 86], [125, 86]],
    [[142, 146], [155, 145], [136, 165], [150, 165]],
    [[95, 102], [106, 101], [88, 119], [99, 118]],
    [[85, 453], [106, 453], [72, 494], [96, 494]],
    [[109, 454], [130, 454], [99, 493], [121, 494]],
    [[135, 454], [155, 454], [124, 495], [146, 495]],
    [[159, 456], [181, 456], [149, 495], [173, 496]],
    [[184, 455], [208, 454], [177, 495], [202, 495]],
    [[212, 455], [235, 455], [205, 495], [228, 495]],

    [[70, 80], [79, 80], [61, 95], [70, 95]],
    [[81, 80], [90, 80], [76, 94], [85, 92]],
    [[94, 80], [103, 80], [89, 91], [98, 91]],
    [[108, 77], [116, 77], [103, 89], [111, 88]],
    [[119, 75], [129, 74], [115, 89], [124, 88]],
    [[132, 74], [141, 74], [127, 87], [137, 87]],
    [[145, 74], [154, 74], [140, 87], [150, 86]],
    [[158, 72], [167, 72], [153, 86], [163, 86]],
    [[171, 72], [181, 72], [167, 86], [176, 86]],
    [[185, 72], [194, 72], [180, 86], [191, 86]],
    [[198, 72], [208, 72], [194, 86], [204, 86]],
    [[211, 72], [220, 72], [208, 85], [218, 85]],
    [[224, 72], [235, 72], [220, 85], [232, 85]],
    [[239, 72], [248, 72], [236, 84], [246, 84]],
    [[260, 70], [270, 70], [258, 83], [267, 83]],
    [[275, 69], [285, 69], [271, 82], [283, 82]],
    [[289, 67], [300, 67], [288, 81], [297, 81]],
    [[304, 66], [314, 66], [301, 81], [311, 81]],
    # [[, ], [, ], [, ], [, ]],
    # [[, ], [, ], [, ], [, ]],
    # [[, ], [, ], [, ], [, ]],
    # [[, ], [, ], [, ], [, ]],
    # [[, ], [, ], [, ], [, ]],
    # [[, ], [, ], [, ], [, ]],
    # [[, ], [, ], [, ], [, ]],

    
    [[105, 15], [115, 15], [99, 32], [109, 32]],
    [[91, 40], [103, 40], [85, 57], [97, 57]],
    [[70, 81], [82, 81], [61, 98], [75, 98]],
    [[59, 104], [71, 105], [48, 125], [61, 125]],
    [[34, 154], [45, 155], [21, 179], [33, 179]],
    [[16, 187], [30, 187], [5, 208], [19, 209]],

    [[13, 252], [25, 252], [2, 275], [15, 275]],

    [[28, 252], [40, 252], [17, 275], [30, 275]], 
    [[45, 251], [57, 251], [34, 274], [47, 274]],
    [[60, 251], [72, 251], [49, 274], [62, 274]], #<-
    [[73, 250], [85, 250], [62, 273], [75, 273]],
    [[88, 250], [100, 250], [77, 273], [90, 273]],
    [[103, 249], [115, 249], [92, 273], [105, 273]],
    [[118, 249], [130, 249], [107, 273], [120, 273]],
    [[133, 248], [145, 248], [122, 273], [135, 273]],
    [[148, 248], [160, 248], [137, 273], [150, 273]],
    [[163, 247], [175, 247], [152, 273], [165, 273]],
    [[178, 247], [190, 247], [167, 273], [180, 273]],
    [[193, 246], [205, 246], [182, 273], [195, 273]],
    [[208, 246], [220, 246], [197, 273], [180, 273]],

    [[222, 248], [235, 248], [218, 272], [231, 272]],
    
    [[240, 248], [254, 248], [237, 272], [250, 272]],
    [[259, 248], [272, 248], [255, 272], [269, 272]],
    [[277, 247], [291, 247], [274, 272], [288, 272]],
    [[295, 247], [309, 247], [293, 272], [307, 272]],
    [[313, 247], [328, 247], [312, 272], [326, 272]],
    [[332, 246], [346, 246], [330, 272], [345, 272]],
    [[350, 246], [365, 246], [349, 272], [363, 272]],
    [[368, 246], [383, 246], [368, 272], [382, 272]],
    [[387, 245], [402, 245], [386, 272], [401, 272]],

    [[405, 245], [420, 245], [405, 272], [420, 272]],

    [[10, 290], [26, 290], [0, 318], [15, 319]],
    [[23, 380], [41, 380], [8, 414], [28, 414]],
    [[17, 451], [37, 451], [0, 497], [22, 497]],
    [[19, 574], [44, 575], [0, 634], [29, 635]],
])

In [75]:
def main():
    template_spaces = np.float32([
        [[2, 0], [17, 0], [1, 22], [17, 22]],
        [[8, 2], [34, 3], [1, 43], [26, 44]],
    ])

    template_files = sorted([f for f in os.listdir(empty_parking_folder) if f.endswith('.jpg') or f.endswith('.png')])
    templates = []
    for i, template_file in enumerate(template_files):
        template_path = os.path.join(empty_parking_folder, template_file)
        template_image = cv2.imread(template_path)    
        template_image = cv2.cvtColor(template_image, cv2.COLOR_BGR2RGB)
        template_image = cv2.GaussianBlur(template_image, (3,3), 0)

        x, y, _ = template_image.shape
        
        dst_points = np.float32([[0, 0], [y, 0], [0, x], [y, x]])

        new_image = perspective_transform(template_image, template_spaces[i], dst_points)
        new_image = cv2.resize(new_image, (50, 80), interpolation=cv2.INTER_CUBIC)
        
        templates.append(new_image)

    frame_files = sorted([f for f in os.listdir(frames_folder) if f.endswith('.jpg') or f.endswith('.png')])

    for idx, frame_file in enumerate(frame_files):

        if idx == 100:
            break
        idx += 1
        
        frame_path = os.path.join(frames_folder, frame_file)
        frame = cv2.imread(frame_path)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        for idx, template in enumerate(templates):
            x, y, _ = template_image.shape
            dst_points = np.float32([[0, 0], [y, 0], [0, x], [y, x]])
                
            for parking_place_coor in parking_spaces:
                frame_copy = perspective_transform(frame, parking_place_coor, dst_points)[0:x, 0:y]
                frame_copy = cv2.resize(frame_copy, (50, 80), interpolation=cv2.INTER_CUBIC)

                # show_in_row([frame_copy, template])
                
                if mse_match(frame_copy, template):
                    parking_place_coor[[0, 1, 2, 3]] = parking_place_coor[[0, 1, 3, 2]]
                    pts = np.array(parking_place_coor, np.int32)
                    parking_place_coor[[0, 1, 2, 3]] = parking_place_coor[[0, 1, 3, 2]]
                    pts = pts.reshape((-1,1,2))
                    cv2.polylines(frame,[pts],True,(255,0,0), 2)
                
                    # show_in_row([frame, frame_copy, template])
        # show_in_row([frame])
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output + '/' + frame_file, frame)

In [76]:
if __name__ == "__main__":
    start_time = time.time()
    main()
    print(f"Working time: {time.time() - start_time} sec.")

Working time: 21.94696068763733 sec.
